In [3]:
import time as time
import datetime as datetime
import pandas as pd
import yfinance as yf
import matplotlib as plt
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [4]:
#Ticker in url
ticker = 'AAPL'

#Timeperiods of data set "Y/M/D/time"
#this will help when selecting the desired dates and will pull the data set from yahoo finance.
period1 = int(time.mktime(datetime.datetime(2020, 1, 2, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2023, 1, 6, 23, 59).timetuple()))
interval = '1d' # 1wk, 1m

#Yahoo Finance url
url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'


In [5]:
#shaping data frame to backtesting requirements
df = pd.read_csv(url)
columns = ['Date', 'Open', 'High', 'Low', 'Close', 'adj close', 'Volume']
df.columns = columns

In [6]:
#index data frame to date time index to fit backtesting.py
#df requirements
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

In [7]:
#dropping columns that are not necessary
df.drop('Date', inplace=True, axis=1)
df.drop('adj close', inplace=True, axis=1)
df

,Open,High,Low,Close,Volume
2020-01-02,74.059998,75.150002,73.797501,75.087502,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,132079200
...,...,...,...,...,...
2022-12-30,128.410004,129.949997,127.430000,129.929993,76960600
2023-01-03,130.279999,130.899994,124.169998,125.070000,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,89113600
2023-01-05,127.129997,127.769997,124.760002,125.019997,80962700


In [8]:
import sys
print(sys.version)

3.10.7 (v3.10.7:6cc6b13308, Sep  5 2022, 14:02:52) [Clang 13.0.0 (clang-1300.0.29.30)]


In [11]:
#the overall trading strategy function
class RSI(Strategy): 
    
    #define upper and lower bands of indicator. If the stock price
    #dips below 30 it buys and above 70 it sells
    upper_bound = 92
    lower_bound = 24
   
    #for stop loss or take profits, [-1] is the most recent price on df
    #price = self.data.Close[-1]


    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, 7)
    
    def next(self):
        
        if crossover(self.rsi, self.upper_bound):
            #if this statement is true the below command signals a sell.
            self.position.close()
            self.sell()
        
        elif crossover(self.lower_bound, self.rsi):
            #buy command 
            #for stop loss(sl) or take profits(tp) use 'self.buy(sl=0.90*price)' 
            #1.15 = 15% profit and 0.95 = -5% loss
            #for buying a certain amount use 'self.buy(size = 0.10)' to buy
            #10% of portfolio
            # self.buy(tp=1.15*price, sl=0.95 price)
            self.position.close()
            self.buy()
                       

#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to 
#backtesting.py on github
bt = Backtest(df, RSI, cash = 10_000)
stats = bt.run()

stats

#the plotting function does not work in python 3.8.7 so it needs to be 
#run in a earlier python like python 3.6 to graph the trades 

#bt.plot()

Start                     2020-01-02 00:00:00
End                       2023-01-06 00:00:00
Duration                   1100 days 00:00:00
Exposure Time [%]                   95.263158
Equity Final [$]                 16228.167217
Equity Peak [$]                  23396.166321
Return [%]                          62.281672
Buy & Hold Return [%]               72.625259
Return (Ann.) [%]                   17.414283
Volatility (Ann.) [%]               44.258271
Sharpe Ratio                          0.39347
Sortino Ratio                        0.702061
Calmar Ratio                         0.558524
Max. Drawdown [%]                  -31.179124
Avg. Drawdown [%]                   -6.222157
Max. Drawdown Duration      368 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   15
Win Rate [%]                             40.0
Best Trade [%]                      92.427891
Worst Trade [%]                    -14.610993
Avg. Trade [%]                    

In [10]:
#'optimize' function finds the optimal ranges of the trading strategy to
#return the maximum value of the desired measurement such as 'Return [%]'
#the range of the variables values must be defined and it will select the 
#optimal value in that range
optim = bt.optimize(upper_bound = range(50,95,7),
                    lower_bound = range(10,45,7),
                    maximize = ('Return [%]'),
                    constraint = lambda param: param.upper_bound > param.lower_bound)

states = bt.plot()
states

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


Row(id='1476', ...)

In [120]:
optim

Start                     2020-01-02 00:00:00
End                       2023-01-06 00:00:00
Duration                   1100 days 00:00:00
Exposure Time [%]                   95.263158
Equity Final [$]                   16256.1271
Equity Peak [$]                  23428.186209
Return [%]                          62.561271
Buy & Hold Return [%]               72.625259
Return (Ann.) [%]                   17.481322
Volatility (Ann.) [%]                44.24908
Sharpe Ratio                         0.395066
Sortino Ratio                        0.705052
Calmar Ratio                         0.561053
Max. Drawdown [%]                  -31.158066
Avg. Drawdown [%]                   -6.219957
Max. Drawdown Duration      368 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   35
Win Rate [%]                        28.571429
Best Trade [%]                      98.443056
Worst Trade [%]                    -12.860054
Avg. Trade [%]                    

In [119]:
#this gives the values of the optimal values used in the strategy
print(optim['_strategy'].lower_bound, optim['_strategy'].upper_bound)

31 92


In [59]:


#the overall trading strategy function
class MACD(Strategy): 
     
    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def init(self):
      
        self.macd, self.macdsignal, self.macdhist = self.I(talib.MACD, self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
    
    def next(self):
        
        if crossover(self.macdsignal, self.macd):
            #if this statement is true the below command signals a sell.
            self.position.close()
            self.sell()
        
        elif crossover(self.macd, self.macdsignal):
            #buy command 
            self.position.close()
            self.buy()
                       

#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to 
#backtesting.py on github
bt = Backtest(df, MACD, cash = 10_000)
stats = bt.run()
stats
#the plotting function does not work in python 3.8.7 so it needs to be 
#run in a earlier python like python 3.6 to graph the trades 
#bt.plot()

Start                     2020-01-02 00:00:00
End                       2023-01-06 00:00:00
Duration                   1100 days 00:00:00
Exposure Time [%]                   95.131579
Equity Final [$]                  8540.759063
Equity Peak [$]                  21607.735074
Return [%]                         -14.592409
Buy & Hold Return [%]              294.157014
Return (Ann.) [%]                   -5.095748
Volatility (Ann.) [%]               65.550147
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -78.97645
Avg. Drawdown [%]                  -19.830276
Max. Drawdown Duration      973 days 00:00:00
Avg. Drawdown Duration      148 days 00:00:00
# Trades                                   59
Win Rate [%]                        33.898305
Best Trade [%]                      52.863243
Worst Trade [%]                    -32.333803
Avg. Trade [%]                    

In [60]:
bt.plot()

Row(id='7239', ...)